# Download and Set-Up Paths

In [10]:
pwd

'/content/EfficientNetDFU'

In [1]:
from __future__ import print_function
from IPython import display
import sys
import os

# Download source code.
if "efficientnet_model.py" not in os.getcwd() or "imagenet.py" not in os.getcwd():
  !git clone --depth 1 https://github.com/rezabasiri/EfficientNetDFU.git
  display.clear_output()
  os.chdir('EfficientNetDFU')
  sys.path.append('.')
  os.chdir('common')
  sys.path.append('.')
  os.chdir('..')
else:
  !git pull

In [9]:
MODEL = 'efficientnet-b0'

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/{m}.tar.gz
    !tar xf {m}.tar.gz
    display.clear_output()
    !rm -rf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model checkpoint in {}'.format(ckpt_path))

Use model in /content/EfficientNetDFU/efficientnet-b0



# Set-Up Data


In [12]:
# Prepare Training Data
!rm -rf *.zip *.tar tfrecord/ val2017/
!mkdir tfrecord
!PYTHONPATH=".:$PYTHONPATH"  python dataset/create_coco_tfrecord.py \
      --image_dir="/content/drive/My Drive/PhDUofT/DFUC2020/DFU_Challenge/MyCode/Try2/Step2/Chess/Images" \
      --object_annotations_file="/content/drive/My Drive/PhDUofT/DFUC2020/DFU_Challenge/MyCode/Try2/Step2/Chess/annotations/instances_Images.json" \
      --output_file_prefix=tfrecord/train \
      --num_shards=15

python3: can't open file 'dataset/create_coco_tfrecord.py': [Errno 2] No such file or directory


In [ ]:
# Prepare Validation Data
!PYTHONPATH=".:$PYTHONPATH"  python dataset/create_coco_tfrecord.py \
      --image_dir="/content/drive/My Drive/PhDUofT/DFUC2020/DFU_Challenge/MyCode/Try2/Step2/ChessVal/ImagesVal" \
      --object_annotations_file="/content/drive/My Drive/PhDUofT/DFUC2020/DFU_Challenge/MyCode/Try2/Step2/ChessVal/annotations/instances_ImagesVal.json" \
      --output_file_prefix=tfrecord/validation \
      --num_shards=15

In [ ]:
file_pattern = 'train-*-of-00015.tfrecord' # Update to match the number of shards for training set
file_patternVal = 'validation-*-of-00015.tfrecord' # Update to match the number of shards for valid set

images_per_epoch = 57 * len(tf.io.gfile.glob('tfrecord/' + file_pattern))
images_per_epoch = images_per_epoch // 8 * 8  # round to 64.

images_per_epochVal = 57 * len(tf.io.gfile.glob('tfrecord/' + file_patternVal))
images_per_epochVal = images_per_epochVal // 8 * 8  # round to 64.

print('images_per_epoch = {}'.format(images_per_epoch))
print('images_per_epochVal = {}'.format(images_per_epochVal))

# Create the Model and Run Train-Eval

In [ ]:
!mkdir model_dir/
!python main.py --mode=train_and_eval \
    --data_dir=tfrecord \
    --model_name={MODEL} \
    --model_dir=model_dir/{MODEL}-finetune \
    --ckpt= {ckpt} \
    --train_batch_size=8 \
    --eval_batch_size=8 --eval_samples={images_per_epochVal}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=2  \
    --hparams="num_classes=2,max_instances_per_image=5,learning_rate=0.001,moving_average_decay=0,mixed_precision=true,use_augmix=True"

In [ ]:
# import eval_ckpt_main as eval_ckpt
# import tensorflow.compat.v1 as tf

# !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/{model_name}.tar.gz -O {model_name}.tar.gz
# !tar xf {model_name}.tar.gz
# ckpt_dir = model_name
# !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt -O labels_map.txt
# labels_map_file = 'labels_map.txt'


# image_files = [image_file]
# eval_driver = eval_ckpt.get_eval_driver(model_name)
# pred_idx, pred_prob = eval_driver.eval_example_images(
#     ckpt_dir, image_files, labels_map_file)

predicted class for image panda.jpg: 
  -> top_0 (82.75%): giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca  
  -> top_1 (1.51%): ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus  
  -> top_2 (0.37%): lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens  
  -> top_3 (0.23%): American black bear, black bear, Ursus americanus, Euarctos americanus  
  -> top_4 (0.17%): brown bear, bruin, Ursus arctos  
